In [61]:
import pandas as pd
import numpy as np

- use_log: 이용 이력 데이터
- cust_m: 회원 정보 데이터 - 탈퇴한 회원을 포함한 모든 회원에 대한 정보 / end date가 비어져있으면 탈퇴하지 않은 회원
- class_m: 회원구분 데이터(종일/주간/야간)
- camp_m: 프로모션 구분 데이터(무료/반액할인/일반권)

In [62]:
ul = pd.read_csv('use_log.csv')
cust_m = pd.read_csv('customer_master.csv')
class_m = pd.read_csv('class_master.csv')
camp_m = pd.read_csv('campaign_master.csv')

In [63]:
display(ul.head(), cust_m.head(), class_m.head(), camp_m.head())

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


- 고객현황 데이터인 cust_m에 class_m 데이터를 결합하여 새로운 데이터프레임을 생성

In [64]:
cust_join = pd.merge(cust_m, class_m, on = 'class', how = 'left')
cust_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000


In [65]:
cust_join = pd.merge(cust_join, camp_m, on = 'campaign_id', how = 'left')
cust_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반


In [66]:
#공통컬럼 기준으로 잘 합쳐졌는지 확인
print(len(cust_m))
print(len(cust_join))

4192
4192


In [67]:
cust_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 0 to 4191
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    4192 non-null   object
 1   name           4192 non-null   object
 2   class          4192 non-null   object
 3   gender         4192 non-null   object
 4   start_date     4192 non-null   object
 5   end_date       1350 non-null   object
 6   campaign_id    4192 non-null   object
 7   is_deleted     4192 non-null   int64 
 8   class_name     4192 non-null   object
 9   price          4192 non-null   int64 
 10  campaign_name  4192 non-null   object
dtypes: int64(2), object(9)
memory usage: 393.0+ KB


In [68]:
cust_join.isna().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

- 원본인 cust_m과 결합된 새로운 데이터프레임인 cust_join의 행의 수가 같으므로 열 기준으로 결합이 잘됨
- 결측치를 확인하는 이유는 결합할 때 키가 없거나 결합이 잘못되면 자동으로 결측치가 들어가기 때문->그러므로 결합 후에는 결측치를 확인해주는게 좋음
- 원래 결측치가 있던 end_data에만 결측치가 있으므로 데이터가 잘 들어가져있다라고 확인됨.

In [69]:
#우리 고객의 수가 4192명이 맞구나를 확인
cust_join['customer_id'].nunique()

4192

- cust_join 데이터프레임 분석
    - 내가 이 데이터프레임을 봤을 때 어떤 궁금한 점을 뽑아낼 수 있을까?
    - 예시
        - 어떤 종류의 회원이 있고 어떤 캠페인으로 많이 가입했는지?
        - 언제 가입/탈퇴한 회원이 많은지?
        - 남녀 비율은 어떤지?
        - 탈퇴한 회원들은 얼만큼의 기간동안 스포츠 센터를 다녔는지?


In [70]:
cust_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반


In [71]:
cust_join.shape

(4192, 11)

In [72]:
cust_join.class_name.value_counts()

0_종일    2045
2_야간    1128
1_주간    1019
Name: class_name, dtype: int64

In [73]:
cust_join.groupby('class_name')['customer_id'].count()

class_name
0_종일    2045
1_주간    1019
2_야간    1128
Name: customer_id, dtype: int64

In [74]:
#회원권 종류별로 비율 확인
#round:반올림하는 함수                                                                  소수점 첫째자리
round((cust_join.groupby('class_name')['customer_id'].count()/cust_join.shape[0])*100, 1)

class_name
0_종일    48.8
1_주간    24.3
2_야간    26.9
Name: customer_id, dtype: float64

In [75]:
#프로모션 현황 비율 확인
#round:반올림하는 함수                                                                  소수점 첫째자리
round((cust_join.groupby('campaign_name')['customer_id'].count()/cust_join.shape[0])*100, 1)

campaign_name
0_입회비반액할인    15.5
1_입회비무료      11.7
2_일반         72.8
Name: customer_id, dtype: float64

In [76]:
#성별현황
cust_join.gender.value_counts()
#cust_join.groupby('gender')['customer_id'].count()

M    2209
F    1983
Name: gender, dtype: int64

In [77]:
#성별현황 비율 확인
round((cust_join.groupby('gender')['customer_id'].count()/cust_join.shape[0])*100, 1)

gender
F    47.3
M    52.7
Name: customer_id, dtype: float64

In [78]:
#탈퇴자 현황
cust_join.is_deleted.value_counts()

0    2842
1    1350
Name: is_deleted, dtype: int64

In [79]:
#탈퇴자 현황 비율 확인
round((cust_join.groupby('is_deleted')['customer_id'].count()/cust_join.shape[0])*100, 1)

is_deleted
0    67.8
1    32.2
Name: customer_id, dtype: float64

- 데이터를 분석하면서 항상 궁금한 점을 생각해보기
    - 남자가 더 많이 가입한 이유가 있을까?
    - 탈퇴한 사람들은 왜 탈퇴했을까?
    - 얘네들은 프로모션만 이용하나?

- 데이터 조건별로 뽑아보기
    - 최신 데이터 뽑아보기
    - 2019년 3월 이용자 뽑기 -> end date가 2019.3.31 이후거나 탈회한 적이 없어야함

In [80]:
cust_join['end_date'] = pd.to_datetime(cust_join['end_date'])
cust_new = cust_join[(cust_join['end_date']>=pd.to_datetime('20190331')) | (cust_join['end_date'].isnull())]
cust_new.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaT,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaT,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaT,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaT,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaT,CA1,0,2_야간,6000,2_일반


In [81]:
cust_new['end_date'].unique()

array([                          'NaT', '2019-03-31T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [82]:
#어떤 회원권을 많이 사용하고 있는지 확인
cust_new.groupby('class_name')['customer_id'].count()

class_name
0_종일    1444
1_주간     696
2_야간     813
Name: customer_id, dtype: int64

In [84]:
round((cust_new.groupby('class_name')['customer_id'].count()/cust_new.shape[0])*100, 1)

class_name
0_종일    48.9
1_주간    23.6
2_야간    27.5
Name: customer_id, dtype: float64

In [86]:
#프로모션 비율 확인
round((cust_new.groupby('campaign_name')['customer_id'].count()/cust_new.shape[0])*100, 1)

campaign_name
0_입회비반액할인    10.5
1_입회비무료       8.2
2_일반         81.3
Name: customer_id, dtype: float64

In [85]:
#성별 비율 확인
round((cust_new.groupby('gender')['customer_id'].count()/cust_new.shape[0])*100, 1)

gender
F    47.4
M    52.6
Name: customer_id, dtype: float64

<데이터 분석 해석>
- 회원권과 성별 변수에 대해서는 전체 데이터에 대해 뽑았던 것과 비율이 크게 다르지 않음
    - 특정 회원권이나 성별이 탈퇴한 것은 아님
- 전체 데이터에서 프로모션 비율 중 일반가입이 70%정도였는데, 최신 사용자 데이터에서는 일반 가입이 80%(약 10% 차이)
    - 프로모션으로 가입한 사람은 일반 가입한 사람보다 계속 이용할 가능성이 낮을 수 있지 않을까?


- 이용이력 데이터 분석

In [87]:
ul.head()

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


- 회원이 스포츠센터를 이용할 때 시간이 찍히기 때문에 시간적 요소를 분석해 볼 수 있겠다.
- 예시
    - 한 달 이용 횟수의 변화
    - 회원이 스포츠센터를 정기적으로 이용하는지
    - 회원들이 월 별 스포츠센터를 이용하는 평균 수치, 중앙값, 최대, 최소값

In [90]:
#데이터를 날짜형 데이터로 변환
ul['usedate'] = pd.to_datetime(ul['usedate'])

In [93]:
ul['년월'] = ul['usedate'].dt.strftime('%Y%m')

In [94]:
ul.head()

,log_id,customer_id,usedate,년월
0,L00000049012330,AS009373,2018-04-01,201804
1,L00000049012331,AS015315,2018-04-01,201804
2,L00000049012332,AS040841,2018-04-01,201804
3,L00000049012333,AS046594,2018-04-01,201804
4,L00000049012334,AS073285,2018-04-01,201804


In [95]:
#월별, 고객 id별 이용횟수 카운트
#                                              인덱스 공간 지움? 지우고 확인해보기
ul_month = ul.groupby(['년월', 'customer_id'], as_index=False).count()

In [97]:
ul_month

,년월,customer_id,log_id,usedate
0,201804,AS002855,4,4
1,201804,AS009013,2,2
2,201804,AS009373,3,3
3,201804,AS015315,6,6
4,201804,AS015739,7,7
...,...,...,...,...
36837,201903,TS995853,8,8
36838,201903,TS998593,8,8
36839,201903,TS999079,3,3
36840,201903,TS999231,6,6


- 월별 고객의 스포츠센터 사용횟수 데이터

In [102]:
#컬럼 이름 변경
ul_month.rename(columns={'log_id' : 'cnt'}, inplace=True)
#usedate 열 삭제
ul_month.drop('usedate', axis=1, inplace=True)
ul_month.head()

,년월,customer_id,cnt
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [105]:
#고객별 월평균 사용횟수
ul_cust = ul_month.groupby('customer_id')['cnt'].agg([np.mean, np.median, np.max, np.min])

#인덱스 리셋 -> customer_id
ul_cust = ul_cust.reset_index(drop=False)
ul_cust.head()

,customer_id,mean,median,amax,amin
0,AS002855,4.500000,5.0,7,2
1,AS008805,4.000000,4.0,8,1
2,AS009013,2.000000,2.0,2,2
3,AS009373,5.083333,5.0,7,3
4,AS015233,7.545455,7.0,11,4


- 정기적으로 스포츠센터를 사용하는 고객인지 확인
    - 매주 같은 요일에 스포츠센터에 왔는지 아닌지로 판단해보기
    - 월이나 요일별로 방문한 횟수의 최대값이 4이상인 요일에 하나라도 있으면 이 회원은 정기 사용자로 판단

In [106]:
#요일 뽑기
#weekday(): 월요일 - 0 ~ 일요일- 6으로 나타내는 함수
ul['weekday'] = ul['usedate'].dt.weekday
ul.head()

,log_id,customer_id,usedate,년월,weekday
0,L00000049012330,AS009373,2018-04-01,201804,6
1,L00000049012331,AS015315,2018-04-01,201804,6
2,L00000049012332,AS040841,2018-04-01,201804,6
3,L00000049012333,AS046594,2018-04-01,201804,6
4,L00000049012334,AS073285,2018-04-01,201804,6


In [108]:
ul_week = ul.groupby(['customer_id', '년월', 'weekday'], as_index=False).count()
ul_week

,customer_id,년월,weekday,log_id,usedate
0,AS002855,201804,5,4,4
1,AS002855,201805,2,1,1
2,AS002855,201805,5,4,4
3,AS002855,201806,5,5,5
4,AS002855,201807,1,1,1
...,...,...,...,...,...
93328,TS999855,201901,1,1,1
93329,TS999855,201901,5,4,4
93330,TS999855,201901,6,1,1
93331,TS999855,201902,5,4,4


In [109]:
ul_week.rename(columns={'log_id' : 'cnt'}, inplace=True)
ul_week

,customer_id,년월,weekday,cnt,usedate
0,AS002855,201804,5,4,4
1,AS002855,201805,2,1,1
2,AS002855,201805,5,4,4
3,AS002855,201806,5,5,5
4,AS002855,201807,1,1,1
...,...,...,...,...,...
93328,TS999855,201901,1,1,1
93329,TS999855,201901,5,4,4
93330,TS999855,201901,6,1,1
93331,TS999855,201902,5,4,4


In [110]:
del ul_week['usedate']
ul_week.head()

,customer_id,년월,weekday,cnt
0,AS002855,201804,5,4
1,AS002855,201805,2,1
2,AS002855,201805,5,4
3,AS002855,201806,5,5
4,AS002855,201807,1,1


- 위 과정을 통해 나온 결과물을 분석해 보면 토요일마다 정기적으로 방문하는 사람인가? 라는 생각을 해볼 수 있다.

- ul_week 데이터프레임을 가지고 cnt 4이상이면, 1 아니면 0인 컬럼 만들기

In [115]:
#방법1: apply lambda
#                                       x가 4이상이면 1의 값을 갖고 아니면 0을 갖는다.
ul_week['flag1'] = ul_week['cnt'].apply(lambda x : 1 if x >= 4 else 0)
ul_week.head()

,customer_id,년월,weekday,cnt,flag1
0,AS002855,201804,5,4,1
1,AS002855,201805,2,1,0
2,AS002855,201805,5,4,1
3,AS002855,201806,5,5,1
4,AS002855,201807,1,1,0


In [116]:
#방법2: 불리언 인덱싱
ul_week['flag2']=0
is_over4 = (ul_week['cnt']>=4)
ul_week.loc[(is_over4), 'flag2'] = 1
ul_week.head()

,customer_id,년월,weekday,cnt,flag1,flag2
0,AS002855,201804,5,4,1,1
1,AS002855,201805,2,1,0,0
2,AS002855,201805,5,4,1,1
3,AS002855,201806,5,5,1,1
4,AS002855,201807,1,1,0,0


In [119]:
#방법3: where문 사용 
ul_week['flag3'] = 0
#조건이 참이면 0이고 거짓이면 1을 넣음
ul_week['flag3'] = ul_week['flag3'].where(ul_week['cnt']<4, 1)
ul_week.head()

,customer_id,년월,weekday,cnt,flag1,flag2,flag3
0,AS002855,201804,5,4,1,1,1
1,AS002855,201805,2,1,0,0,0
2,AS002855,201805,5,4,1,1,1
3,AS002855,201806,5,5,1,1,1
4,AS002855,201807,1,1,0,0,0
